In [1]:
import os
import io
import glob
import base64
import sqlite3
import datetime
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
from PIL import Image
import matplotlib as mpl
import japanize_matplotlib
import matplotlib.pyplot as plt
from IPython import display as dd
from matplotlib.font_manager import FontProperties

In [2]:
# Get current time
current_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# Create output folder with the current time as part of the name
folder_name = f"./Output_{current_time}"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 出力
Heatmap_path = folder_name + '/heatmap.png'
Network_path = folder_name + '/network.png'
Plotpoint_path = folder_name + '/plotpoint.png'
GIF_path_100 = folder_name + '/graph_100.gif'
GIF_path_1000 = folder_name + '/graph_1000.gif'

In [3]:
import sqlite3

def SQL_SetUp():
    conn = sqlite3.connect('database.sqlite')
    cursor = conn.cursor()

    # データベースファイルが存在する場合は処理を終了
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    existing_tables = cursor.fetchall()
    if existing_tables:
        print("Database already exists. Exiting setup.")
        conn.close()
        return

    # テーブルを作成
    cursor.execute('''
        CREATE TABLE knowledge (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            angle FLOAT,
            distance FLOAT,
            activation FLOAT DEFAULT 0,
            description TEXT
        )
    ''')

    data = [
        ("直進", 333.44, 1.12, 0),#１番の知識
        ("左寄りの直進", 345.97, 1.03, 0),#２番の知識
        ("右寄りの直進", 0, 1, 0),#１番の知識
        ("右寄りの直進", 14.04, 1.03, 0),#１番の知識
        ("直進", 26.56, 1.12, 0)#１番の知識
    ]

    # データを挿入
    cursor.executemany("INSERT INTO knowledge (description, angle, distance, activation) VALUES (?, ?, ?, ?)", data)

    conn.commit()
    conn.close()

# データベースのセットアップ
SQL_SetUp()


Database already exists. Exiting setup.


In [4]:
def SQL_GetData():
    conn = sqlite3.connect('database.sqlite')
    cursor = conn.cursor()

    # データを取得してDataFrameに格納
    cursor.execute("SELECT * FROM knowledge")
    rows = cursor.fetchall()
    columns = [description[0] for description in cursor.description]
    df = pd.DataFrame(rows, columns=columns)

    conn.close()

    return df

# データの取得と表示
df = SQL_GetData()
print(df)


   id   angle  distance  activation description
0   1  333.44      1.12         0.0          直進
1   2  345.97      1.03         0.0      左寄りの直進
2   3    0.00      1.00         0.0      右寄りの直進
3   4   14.04      1.03         0.0      右寄りの直進
4   5   26.56      1.12         0.0          直進


In [5]:
# 新しいデータを追加
new_id = len(df) + 1
new_activation = -1.0
new_description = '障害物'
new_angle = float(input('角度を入力してください: '))
new_distance = float(input('距離を入力してください: '))

new_data = pd.DataFrame({
    'id': [new_id],
    'activation': [new_activation],
    'description': [new_description],
    'angle': [new_angle],
    'distance': [new_distance]
})

input_df = pd.concat([df, new_data], ignore_index=True)

# idとdescriptionを結合した文字列を作成
input_df['id_description'] = input_df['id'].astype(str) + '_' + input_df['description']

# ベクトルとして角度と距離を使用するため、データを準備
vectors = input_df[['angle', 'distance']]

# ベクトル間のユークリッド距離を計算
distances = np.linalg.norm(vectors.values[:, np.newaxis] - vectors.values, axis=2)

# クロス表に距離を格納
cross_table = pd.DataFrame(distances, index=input_df['id_description'], columns=input_df['id_description'])

# ユークリッド距離を求めたcross_table
#print(cross_table)

# ユークリッド距離の評価指標を計算し、再度DataFrameに格納
max_distance = np.nanmax(cross_table.values)  # ユークリッド距離の最大値（NaNを除く）
evaluated_values = 1 - cross_table.values / max_distance
activation_table = pd.DataFrame(evaluated_values, index=cross_table.index, columns=cross_table.columns)

# ユークリッド距離を正規化した値をdfに格納
#print(activation_table)

# 評価指標を1/10にスケーリング
activation_table_div10 = activation_table / 10
activation_table_min1 = 1-activation_table

# 再度activation_tableを表示
print(activation_table_div10)
print(activation_table_min1)

input_df2=input_df.copy()

print(input_df)
print(input_df2)



id_description      1_直進  2_左寄りの直進  3_右寄りの直進  4_右寄りの直進      5_直進     6_障害物
id_description                                                            
1_直進            0.100000  0.096378  0.003622  0.007680  0.011299  0.096027
2_左寄りの直進        0.096378  0.100000  0.000000  0.004058  0.007677  0.092445
3_右寄りの直進        0.003622  0.000000  0.100000  0.095942  0.092323  0.007502
4_右寄りの直進        0.007680  0.004058  0.095942  0.100000  0.096381  0.011560
5_直進            0.011299  0.007677  0.092323  0.096381  0.100000  0.015179
6_障害物           0.096027  0.092445  0.007502  0.011560  0.015179  0.100000
id_description      1_直進  2_左寄りの直進  3_右寄りの直進  4_右寄りの直進      5_直進     6_障害物
id_description                                                            
1_直進            0.000000  0.036218  0.963783  0.923201  0.887013  0.039729
2_左寄りの直進        0.036218  0.000000  1.000000  0.959418  0.923230  0.075554
3_右寄りの直進        0.963783  1.000000  0.000000  0.040582  0.076770  0.924976
4_右寄りの直進        0.923201 

In [18]:

for i in range(len(activation_table_div10.columns)-1):
    activity_value_temp = activation_table_div10.loc[activation_table_div10.columns[i], activation_table_div10.columns[-1]]
    input_df2.loc[input_df2['id_description'] == activation_table_div10.columns[i], 'activation'] += activity_value_temp

# 画像を格納するリスト
images = []

while not (input_df2['activation'] > 1).any():
    for i in range(len(activation_table_div10.columns)):
        for j in range(len(activation_table_div10.columns)):
            if i==j:
                continue
            activity_value_temp = activation_table_div10.loc[activation_table_div10.columns[i], activation_table_div10.columns[j]]
            input_df2.loc[input_df2['id_description'] == activation_table_div10.columns[i], 'activation'] += activity_value_temp

# descriptionの最大値を持つレコードを出力
max_description = input_df2['activation'].max()
max_records = input_df2[input_df2['activation'] == max_description]
print(max_records['id_description'][0])

1_直進
